## Problem Set 1 - BDMLAE

In [1]:
#Packages:
import pandas as pd
import numpy as np
import pyreadr as pyr
import sklearn as sk
import scipy as sc
from scipy.stats import bootstrap
from sklearn.impute import KNNImputer

In [2]:
#Setting work space and master data frame:
bd=pyr.read_r("C:/Users/ASUS/Documents/Universidad de los Andes/2022-2/Big Data and Machine Learning for Applied Economics/Taller 1/PS1_BD-ML/bdPS1.Rdata")
print(bd.keys()) #Load database.
df0=bd["df"]
df=df0.drop(df0[df0.age<18].index) #Restrict the data to legal age individuals (age>=18).
df=df.drop(df[df.ocu==0].index) #Restrict the data to employed individuals (ocu==1).
df=(df[["age", "clase", "maxEducLevel", "sizeFirm", "oficio", "hoursWorkUsual", "formal", "ingtot", "mes", "estrato1", "sex", 
"regSalud", "p6426", "p7090","p7510s7a1", "impa", "isa", "impaes", "isaes", "fex_c", "fweight", "relab"]]) #Relevant variables selection. 

odict_keys(['df'])


In [3]:
#Convert categorical variables to dummy variables:
oficio_d=pd.get_dummies(df["oficio"], prefix="oficio") 
sizeFirm_d=pd.get_dummies(df["sizeFirm"], prefix="size") 
estrato1_d=pd.get_dummies(df["estrato1"], prefix="estrato") 
regSalud_d=pd.get_dummies(df["regSalud"], prefix="regs") 
maxeduc_d=pd.get_dummies(df["maxEducLevel"], prefix="educ") 
relab_d=pd.get_dummies(df["relab"], prefix="relab") 

#Merge dummy's variables data frame with master data frame:
df=pd.merge(df, oficio_d, left_index=True, right_index=True) 
df=pd.merge(df, sizeFirm_d, left_index=True, right_index=True)
df=pd.merge(df, estrato1_d, left_index=True, right_index=True)
df=pd.merge(df, regSalud_d, left_index=True, right_index=True)
df=pd.merge(df, maxeduc_d, left_index=True, right_index=True)
df=pd.merge(df, relab_d, left_index=True, right_index=True)
df["p7090"]=np.where(df["p7090"]==2, 0, 1)

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [4]:
#Transform categorial atribute variables to numerical variable: 
df["hoursWorkUsual"]=pd.to_numeric(df["hoursWorkUsual"])
df["formal"]=pd.to_numeric(df["formal"])
df["p6426"]=pd.to_numeric(df["p6426"])

#Descriptive statistics data frame:
ds=(df[["ingtot", "age", "hoursWorkUsual", "p6426", "clase", "sex", "formal", "p7090", "estrato_1", "estrato_2", "estrato_3", "estrato_4", 
"estrato_5", "estrato_6", "regs_1", "regs_2", "regs_3", "educ_1", "educ_3", "educ_4", "educ_5", "educ_6", "educ_7"]].describe(include="all"))
ds=ds.T
ds=ds[["count", "mean", "std", "min", "50%", "max"]]
print(ds.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &    count &          mean &           std &   min &           50\% &           max \\
\midrule
ingtot         &  16542.0 &  1.769379e+06 &  2.675628e+06 &   0.0 &  1.051160e+06 &  8.583333e+07 \\
age            &  16542.0 &  3.943616e+01 &  1.348284e+01 &  18.0 &  3.800000e+01 &  9.400000e+01 \\
hoursWorkUsual &  16542.0 &  4.700774e+01 &  1.554274e+01 &   1.0 &  4.800000e+01 &  1.300000e+02 \\
p6426          &  16542.0 &  6.375795e+01 &  8.948775e+01 &   0.0 &  2.400000e+01 &  7.200000e+02 \\
clase          &  16542.0 &  1.000000e+00 &  0.000000e+00 &   1.0 &  1.000000e+00 &  1.000000e+00 \\
sex            &  16542.0 &  5.299843e-01 &  4.991152e-01 &   0.0 &  1.000000e+00 &  1.000000e+00 \\
formal         &  16542.0 &  5.868093e-01 &  4.924214e-01 &   0.0 &  1.000000e+00 &  1.000000e+00 \\
p7090          &  16542.0 &  9.684440e-02 &  2.957547e-01 &   0.0 &  0.000000e+00 &  1.000000e+00 \\
estrato\_1      &  16542.0 &  1.071817e-01 &  3.093535e-01 

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11856\1440160681.py:11: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(ds.to_latex())


In [ ]:
#Missing values:
#imputer=KNNImputer(n_neighbors=10)
#imputer.fit(df)
#df=pd.DataFrame(imputer.transform(df))